# Title: Amongst seniors in Hungary and Switzerland, who has healthier hearts?

## Introduction (Kevin)
- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question


*INTRO TEXT GOES HERE*

## Preliminary exploratory data analysis (Chloe)
- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [3]:
#PRELIMINARY EXPLORATORY DATA ANALYSIS CODE HERE

## Methods (Tom):
- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results

*METHODS TEXT GOES SURE*

## Expected outcomes and significance (Michael):
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?

*EXPECTED OUTCOMES TEXT GOES HERE*